### Environment Setup

In [ ]:
%pip install -q -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [1]:
from dotenv import load_dotenv
_ = load_dotenv()

In the Basics notebook, we created templates and invoked them and finally feed the result to the LLM. We can make this easier by using Langchain expression language which is very similar to the piping in the Linux shells.

In [2]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o", temperature=0)

In [3]:
out = llm.invoke("Hello, who are you?")
print(out.content)

Hello! I'm an AI language model created by OpenAI, here to help answer your questions and provide information on a wide range of topics. How can I assist you today?


### Tools

Tools can be passed to chat models that support tool calling allowing the model to request the execution of a specific function with specific inputs if it decides to use it. A tool can be a very simple function and also complex functions that can be used to perform complex operations.  
`@tool` decorator is used to create tools that can be used in the model call.

##### Defining

In [44]:
from langchain_core.tools import tool

@tool
def multiply(a: int, b: int) -> int:
   """Multiply two numbers."""
   return a * b

In [45]:
print(f"{multiply.name=}")
print(f"{multiply.description=}")
print(f"{multiply.args=}")


multiply.name='multiply'
multiply.description='Multiply two numbers.'
multiply.args={'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}


In [46]:
# self test
multiply.invoke({'a':2, 'b':3})

6

A more complex tool can be defined as the following

In [ ]:
%pip install -q yfinance

In [55]:
import yfinance as yf

@tool
def get_stock_price(symbol):
    """Get the latest price of stock 'symbol'"""
    ticker = yf.Ticker(symbol)
    todays_data = ticker.history(period='1d')
    return float(todays_data.iloc[0]['Close'])

In [56]:
get_stock_price.invoke({'symbol':"AAPL"})

245.0

#### Binding

We can bind the tools to the model in three ways:

1. In constructor with kwargs
2. via invoke
3. After construction with bind_tools

Constructor with kwargs

We can pass the tools to the underlying LLM model, but the tool should have exactly the same structure as LLM expects. Therefore we need to wrap the tool using convert_to_openai_tool.

In [68]:
# Instantiate the model with the tool
from langchain_core.utils.function_calling import convert_to_openai_tool
openai_tools = [convert_to_openai_tool(get_stock_price)]

llm_with_default_tools = ChatOpenAI(model_kwargs={'tools':openai_tools})

In [69]:
llm_with_default_tools.invoke("What is the price of GOOGLE stock?")

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_4JGbER1Sx835CN5apPTCZnK4', 'function': {'arguments': '{"symbol":"GOOGLE"}', 'name': 'get_stock_price'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 56, 'total_tokens': 73, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-baa9dae5-4eef-4071-be86-7a6c0d075808-0', tool_calls=[{'name': 'get_stock_price', 'args': {'symbol': 'GOOGLE'}, 'id': 'call_4JGbER1Sx835CN5apPTCZnK4', 'type': 'tool_call'}], usage_metadata={'input_tokens': 56, 'output_tokens': 17, 'total_tokens': 73, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

Passing the tool to the model via invoke

In [70]:
llm.invoke("What is the price of GOOGLE stock?", tools=openai_tools)

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_epWeegFakjAgaHwPmCD2J25B', 'function': {'arguments': '{"symbol":"GOOGL"}', 'name': 'get_stock_price'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 55, 'total_tokens': 73, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_d28bcae782', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-b8e62292-8f14-4e7e-93fe-f627834f9b4f-0', tool_calls=[{'name': 'get_stock_price', 'args': {'symbol': 'GOOGL'}, 'id': 'call_epWeegFakjAgaHwPmCD2J25B', 'type': 'tool_call'}], usage_metadata={'input_tokens': 55, 'output_tokens': 18, 'total_tokens': 73, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasonin

Easiest, model independent, and most flexible way is to use bind_tools

In [71]:
llm_with_binded_tools = llm.bind_tools([get_stock_price])

In [73]:
out = llm_with_binded_tools.invoke("What is the price of GOOGLE stock?")
out

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_8qFzFshu4b2dBamzNM8ViUYW', 'function': {'arguments': '{"symbol":"GOOGL"}', 'name': 'get_stock_price'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 55, 'total_tokens': 73, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_d28bcae782', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-517114b3-99e9-4559-a281-189b9bb5f9b4-0', tool_calls=[{'name': 'get_stock_price', 'args': {'symbol': 'GOOGL'}, 'id': 'call_8qFzFshu4b2dBamzNM8ViUYW', 'type': 'tool_call'}], usage_metadata={'input_tokens': 55, 'output_tokens': 18, 'total_tokens': 73, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasonin

As seen model succesfully returned the hint to call the tool with the correct parameters.

In [74]:
out.tool_calls

[{'name': 'get_stock_price',
  'args': {'symbol': 'GOOGL'},
  'id': 'call_8qFzFshu4b2dBamzNM8ViUYW',
  'type': 'tool_call'}]

If we asked a question that the model doesn't decide to use a tool, the tool_calls will be empty.

In [77]:
out = llm_with_binded_tools.invoke("What is wheather in New York?")

In [79]:
out

AIMessage(content="I currently don't have access to real-time weather data. You can check a reliable weather website or app for the latest weather updates in New York.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 31, 'prompt_tokens': 55, 'total_tokens': 86, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_d28bcae782', 'finish_reason': 'stop', 'logprobs': None}, id='run-8d7dc6f8-df11-41cf-897a-0c8ec62990a2-0', usage_metadata={'input_tokens': 55, 'output_tokens': 31, 'total_tokens': 86, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

#### Executing

Tool calling is not useful without executing the tool. Simply the args in the result of tool_calls of model response can be passed to the tool to get the result.

In [80]:
out = llm_with_binded_tools.invoke("What is the price of GOOGLE stock?")
out

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_w23z2R4OvWPWO9T920vTN8NF', 'function': {'arguments': '{"symbol":"GOOGL"}', 'name': 'get_stock_price'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 55, 'total_tokens': 73, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_5f20662549', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-dfc92778-6125-4152-bbfe-e85b92ad4a7f-0', tool_calls=[{'name': 'get_stock_price', 'args': {'symbol': 'GOOGL'}, 'id': 'call_w23z2R4OvWPWO9T920vTN8NF', 'type': 'tool_call'}], usage_metadata={'input_tokens': 55, 'output_tokens': 18, 'total_tokens': 73, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasonin

In [82]:
get_stock_price.invoke(out.tool_calls[0]['args'])

196.8699951171875

#### End2End

In a real-world scenario, we can use the tools to perform complex operations and feed the result to the model and model will responed with human friendly message considering tool result.  
This can be accomplished manually, builtin-agents, chains and graphs. Graphs will be introduced in the next notebooks.

##### Very Manual

In [125]:
# define the toolset
tools = [get_stock_price, multiply]

In [126]:
# start conversation
from langchain_core.messages import HumanMessage, ToolMessage

messages = [HumanMessage("Hello, what is the last price of Google stock?")]
out = llm_with_binded_tools.invoke(messages)
# add returned AI message to the conversation
messages.append(out)
out

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_Np8eoxduB9bG1c6lQpzVPFrU', 'function': {'arguments': '{"symbol":"GOOGL"}', 'name': 'get_stock_price'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 58, 'total_tokens': 75, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_5f20662549', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-c38cbb26-3d1c-4693-ae91-d0ee7a5275a0-0', tool_calls=[{'name': 'get_stock_price', 'args': {'symbol': 'GOOGL'}, 'id': 'call_Np8eoxduB9bG1c6lQpzVPFrU', 'type': 'tool_call'}], usage_metadata={'input_tokens': 58, 'output_tokens': 17, 'total_tokens': 75, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasonin

In [127]:
# get call details
tc = out.tool_calls[0]
tc

{'name': 'get_stock_price',
 'args': {'symbol': 'GOOGL'},
 'id': 'call_Np8eoxduB9bG1c6lQpzVPFrU',
 'type': 'tool_call'}

In [128]:
# find relevant langchain wrapped tool and invoke it. tc can be passed as is so that it returns a ToolMessage object just as we need.
# otherwise if you pass args, you'll just get the return value of the tool.
tool_call_result = {tool.name: tool for tool in tools}[tc['name']].invoke(tc)
tool_call_result

ToolMessage(content='196.8699951171875', name='get_stock_price', tool_call_id='call_Np8eoxduB9bG1c6lQpzVPFrU')

In [129]:
# add the tool result to the conversation and call LLM again
messages.append(tool_call_result)
messages


[HumanMessage(content='Hello, what is the last price of Google stock?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_Np8eoxduB9bG1c6lQpzVPFrU', 'function': {'arguments': '{"symbol":"GOOGL"}', 'name': 'get_stock_price'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 58, 'total_tokens': 75, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_5f20662549', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-c38cbb26-3d1c-4693-ae91-d0ee7a5275a0-0', tool_calls=[{'name': 'get_stock_price', 'args': {'symbol': 'GOOGL'}, 'id': 'call_Np8eoxduB9bG1c6lQpzVPFrU', 'type': 'tool_call'}], usage_metadata={'input_tokens': 58, 'output_tokens': 17, 't

In [130]:
llm_with_binded_tools.invoke(messages)

AIMessage(content='The latest price of Google (GOOGL) stock is $196.87.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 91, 'total_tokens': 110, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_d28bcae782', 'finish_reason': 'stop', 'logprobs': None}, id='run-38262798-2d72-4860-9675-01adb73c30f4-0', usage_metadata={'input_tokens': 91, 'output_tokens': 19, 'total_tokens': 110, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

##### Less Manual

Sometimes LLM model may need multiple calls to the tool to get the desired result. Suppose we ask stock price of multiple stocks.
In the less manual way, we'll need to call the tool with batch and feed the result to the model and we'll do this with a ChatPromptTemplate that contains messages placeholder.

In [137]:
# we can define a template that can be used to contain both user input and other messages (AI, Tool)
# messages variable can be empty for the first call and later it can be filled with AI and Tool messages.
prompt_template = ChatPromptTemplate(
    [
        ("human", "{user_input}"),
        ("placeholder", "{messages}"),
    ]
)

In [138]:
# create a chain from template to LLM with tools
templated_llm_with_binded_tools = prompt_template | llm_with_binded_tools

In [140]:
user_input = "What is the price of GOOGLE and APPL stocks?"
ai_message = templated_llm_with_binded_tools.invoke(user_input)
ai_message.tool_calls


[{'name': 'get_stock_price',
  'args': {'symbol': 'GOOGL'},
  'id': 'call_6g7e7y4RmhyrH2BDDSjdXBLQ',
  'type': 'tool_call'},
 {'name': 'get_stock_price',
  'args': {'symbol': 'AAPL'},
  'id': 'call_37ti9SrhmZhPa6lZ8lFBJbRP',
  'type': 'tool_call'}]

As seen above, we have multiple tool_call requests.

In [135]:
tool_msgs = get_stock_price.batch(ai_message.tool_calls)
tool_msgs

[ToolMessage(content='196.8699951171875', name='get_stock_price', tool_call_id='call_eecrNjq3XV6wnPyof4Dxf2aA'),
 ToolMessage(content='245.0', name='get_stock_price', tool_call_id='call_1ZXNYTNTqEIZIoW6J7WxsTJW')]

In [136]:
templated_llm_with_binded_tools.invoke({"user_input":user_input, "messages":[ai_message, *tool_msgs]})

AIMessage(content='The current stock price for Google (GOOGL) is $196.87, and for Apple (AAPL) is $245.00.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 32, 'prompt_tokens': 131, 'total_tokens': 163, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_5f20662549', 'finish_reason': 'stop', 'logprobs': None}, id='run-2ed0d7d0-6cca-4e03-94a6-35c3ffd10474-0', usage_metadata={'input_tokens': 131, 'output_tokens': 32, 'total_tokens': 163, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

##### AgentExecutor

In [112]:
from langchain_core.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_messages([
    ("human", "{input}"), 
    ("placeholder", "{agent_scratchpad}"),
])

In [113]:
from langchain.agents import create_tool_calling_agent, AgentExecutor

agent = create_tool_calling_agent(llm, tools, prompt_template)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

agent_executor.invoke({"input":"what's Google stock price"})



> Entering new AgentExecutor chain...

Invoking: `get_stock_price` with `{'symbol': 'GOOGL'}`


196.8699951171875The current stock price of Google (GOOGL) is $196.87.

> Finished chain.


{'input': "what's Google stock price",
 'output': 'The current stock price of Google (GOOGL) is $196.87.'}

##### Chain

Using chaining with @chain decator, we can even simplify the scenario in `Less Manual` section.

In [142]:
import datetime
from langchain_core.runnables import RunnableConfig, chain

# again we'll use a template that can contain messages
today = datetime.datetime.today().strftime("%D")
prompt_template = ChatPromptTemplate(
    [
        ("human", "{user_input}"),
        ("placeholder", "{messages}"),
    ]
)

In [144]:
# define the chain to combine user input, ai messages and tool messages
@chain
def tool_chain(user_input: str, config: RunnableConfig):
    input_ = {"user_input": user_input}
    ai_msg = templated_llm_with_binded_tools.invoke(input_, config=config)
    tool_msgs = get_stock_price.batch(ai_msg.tool_calls, config=config)
    return templated_llm_with_binded_tools.invoke({**input_, "messages": [ai_msg, *tool_msgs]}, config=config)

In [145]:
# invoke the chain
tool_chain.invoke("What is the price of GOOGLE and APPL stocks?")

AIMessage(content='The current stock price for Google (GOOGL) is $196.87, and for Apple (AAPL) is $245.00.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 32, 'prompt_tokens': 131, 'total_tokens': 163, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_5f20662549', 'finish_reason': 'stop', 'logprobs': None}, id='run-02080b5f-c224-4f07-adfc-80413552bf4e-0', usage_metadata={'input_tokens': 131, 'output_tokens': 32, 'total_tokens': 163, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

#### Builtin Tools

There are many builtin tools under langchain-community packpage. These tools can be used in the model call.   
https://python.langchain.com/docs/integrations/tools/

In [114]:
%pip install -q duckduckgo-search

Note: you may need to restart the kernel to use updated packages.


In [116]:
from langchain_community.tools import DuckDuckGoSearchRun

search = DuckDuckGoSearchRun()

search.invoke({'query':"Obama's first name?"})

"The White House, official residence of the president of the United States, in July 2008. The president of the United States is the head of state and head of government of the United States, [1] indirectly elected to a four-year term via the Electoral College. [2] The officeholder leads the executive branch of the federal government and is the commander-in-chief of the United States Armed ... Obama's father, Barack Obama, Sr., was a teenage goatherd in rural Kenya, won a scholarship to study in the United States, and eventually became a senior economist in the Kenyan government.Obama's mother, S. Ann Dunham, grew up in Kansas, Texas, and Washington state before her family settled in Honolulu.In 1960 she and Barack Sr. met in a Russian language class at the University of Hawaii ... Here is a list of the presidents and vice presidents of the United States along with their parties and dates in office. ... Chester A Arthur: Twenty-First President of the United States. 10 Interesting Facts 

### Structured Outputs